In [20]:
import glob, os
TEST_DATA_FOLDERS = [
    "/n/sd7/trung/csp/data/erica/annotation/test/interview",
    "/n/sd7/trung/csp/data/erica/annotation/test/dating",
    "/n/sd7/trung/csp/data/erica/annotation/test/attentive"
]
TRAIN_DATA_FOLDERS = [
    "/n/sd7/trung/csp/data/erica/annotation/kyoto17/spring/interview",
    "/n/sd7/trung/csp/data/erica/annotation/kyoto17/spring/dating",
    "/n/sd7/trung/csp/data/erica/annotation/kyoto17/spring/attentive",
    #"/n/sd7/trung/csp/data/erica/annotation/kyoto16/interview",
    #"/n/sd7/trung/csp/data/erica/annotation/kyoto16/dating",
    #"/n/sd7/trung/csp/data/erica/annotation/kyoto16/attentive",
    #"/n/sd7/trung/csp/data/erica/annotation/kyoto16/labintro",
]

mode = "train"

from subprocess import call
from struct import unpack, pack
import numpy as np

import IPython
import wave
from pydub import AudioSegment
import re

import MeCab
mt = MeCab.Tagger("-Owakati")
mt.parse('')

OUTPUT_FOLDER = "/n/sd7/trung/csp/data/erica"

In [25]:
mean = open("/n/sd7/trung/csp/data/aps-sps/mean.dat").read().split('\n')[:-1]
mean = np.array([float(x) for x in mean])
var = open("/n/sd7/trung/csp/data/aps-sps/var.dat").read().split('\n')[:-1]
var = np.array([float(x) for x in var])

def get_features(infile, outfile):
    call([
        "/n/sd7/trung/bin/htk/HTKTools/HCopy",
        "-C", "/n/sd7/trung/config.lmfb.40ch", "-T", "1",
        infile, outfile
    ])
    fh = open(outfile, "rb")
    spam = fh.read(12)
    nSamples, sampPeriod, sampSize, parmKind = unpack(">IIHH", spam)
    veclen = int(sampSize / 4)
    fh.seek(12, 0)
    dat = np.fromfile(fh, dtype=np.float32)
    dat = dat.reshape(len(dat) // veclen, veclen)
    dat = dat.byteswap()
    fh.close()
    
    dat = (dat - mean) / np.sqrt(var)
    fh = open(outfile, "wb")
    fh.write(spam)
    fh.seek(12, 0)
    dat.tofile(fh)
    fh.close()
    return dat

def get_features_npy(infile, outfile):
    call([
        "/n/sd7/trung/bin/htk/HTKTools/HCopy",
        "-C", "/n/sd7/trung/config.lmfb.40ch", "-T", "1",
        infile, outfile
    ])
    fh = open(outfile, "rb")
    spam = fh.read(12)
    nSamples, sampPeriod, sampSize, parmKind = unpack(">IIHH", spam)
    veclen = int(sampSize / 4)
    fh.seek(12, 0)
    dat = np.fromfile(fh, dtype=np.float32)
    dat = dat.reshape(len(dat) // veclen, veclen)
    dat = dat.byteswap()
    fh.close()
    if len(dat) > 1000: return False
    
    dat = (dat - mean) / np.sqrt(var)
    np.save(outfile, dat)

In [28]:
filepaths = []
targets = []

targets = []
oov = {}
for DATA_FOLDER in TEST_DATA_FOLDERS if mode == "test" else TRAIN_DATA_FOLDERS:
    for file in glob.glob(os.path.join(DATA_FOLDER, "*.txt")):
        #print(file)
        date, id, obj = os.path.basename(file)[:-4].split('_')[:3]
        longid = date + "_" + id
        year = date[:4]
        if longid != "20161207_04": continue
        
        encoding = 'shift-jis'
        if longid in ['20161205_03', '20161209_05', '20161215_04', '20161208_03', '20161205_07', '20161212_01', '20161209_04', 
                      '20161209_03', '20161208_01', '20161212_03', '20161208_02', '20161207_05', '20161215_03', '20161207_04',
                     '20161205_04']: encoding = 'utf-8'
        
        lines = open(file, encoding=encoding).read().split('\n')
        # print(lines)

        wavpath = '/n/sd7/trung/csp/data/erica/dialogue/%s/%s/%s/%s_audio_mix.wav' % (year, date, longid, longid)
        wav = AudioSegment.from_wav(wavpath)
        if not os.path.exists(os.path.join(OUTPUT_FOLDER, "wav", longid)):
            os.mkdir(os.path.join(OUTPUT_FOLDER, "wav", longid))
        if not os.path.exists(os.path.join(OUTPUT_FOLDER, "htk", longid)):
            os.mkdir(os.path.join(OUTPUT_FOLDER, "htk", longid))
        if not os.path.exists(os.path.join(OUTPUT_FOLDER, "npy", longid)):
            os.mkdir(os.path.join(OUTPUT_FOLDER, "npy", longid))

        k = 0
        while k < len(lines):
            line = lines[k].strip()
            if longid == '20161209_05':
                start, end = line.split(' ')[1:3]
            else:
                start, end = line.split(' ')[1].split('-')
            start, end = float(start) * 1000, float(end) * 1000
            utt_id = line.split(' ')[0]

            output_wav = os.path.join(OUTPUT_FOLDER, "wav", longid, "%s_%s.wav" % (utt_id, obj))
            output_htk = os.path.join(OUTPUT_FOLDER, "htk", longid, "%s_%s.htk" % (utt_id, obj))
            output_npy = os.path.join(OUTPUT_FOLDER, "npy", longid, "%s_%s.npy" % (utt_id, obj))
            utt = AudioSegment.silent(500) + wav[start:end] + AudioSegment.silent(500)
            utt.export(output_wav, format='wav', bitrate=32000)
            flg = False
            if get_features_npy(output_wav, output_npy) == False:
                print(line)
                flg = True
            filepaths.append(output_npy)

            if line[-2:] == obj + ':':
                k += 1
                while k < len(lines):
                    line = lines[k].strip()
                    if flg: print(line)
                    if line[-2:] == obj + ':': break
                    k += 1
            else: k += 1

0074 00209.631-00310.054 U:
(F へー)/L/oth %終了時刻間違っていないか?
0075 00210.814-00212.113 U:


In [11]:
print("Utterences:", len(filepaths))

Utterences: 0
